<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/zindi/landslide_prevention/notebooks/02_xgboost_thresholding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%%capture
!pip install --upgrade xgboost
!pip install --upgrade optuna

In [ ]:
import os
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4)

import xgboost
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [ ]:
#remove cell to run future versions
assert optuna.__version__ == '3.0.2', f'Change in Optuna version. Original notebook version: 3.0.2'
assert xgboost.__version__ == '1.6.2', f'Change in XGBoost version. Original notebook version: 1.6.2'

In [ ]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    HAVE_GPU = True
except Exception:
    HAVE_GPU = False

print(f'GPU available: {HAVE_GPU}')

GPU available: True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preparation

In [ ]:
DATA_PATH = '/content/drive/MyDrive/data_science_competitions/zindi/landslide_prevention/data'

train = pd.read_csv(f'{DATA_PATH}/raw/train.csv')
test = pd.read_csv(f'{DATA_PATH}/raw/test.csv')
sample_sub = pd.read_csv(f'{DATA_PATH}/raw/sample_sub.csv')

train_agg = pd.read_csv(f'{DATA_PATH}/processed/train_agg.csv')
test_agg = pd.read_csv(f'{DATA_PATH}/processed/test_agg.csv')

In [ ]:
TARGET = train['Label']

In [ ]:
cat_features = [f for f in test.columns if f.endswith('geology')]
cat_features.extend(['agg_geology_mode', 'agg_geology_nunique'])

In [ ]:
def preprocess(df: pd.DataFrame, is_train: bool = False) -> pd.DataFrame:
    
    df = df.drop('Sample_ID', axis=1)
    if is_train:
        df = df.drop('Label', axis=1)
    
    #reduce memory usage
    def reduce_mem(df: pd.DataFrame) -> pd.DataFrame:
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in ['int16', 'int32', 'int64', 'float32', 'float64']:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type).startswith('int'):
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        return df

    df = reduce_mem(df)

    for col in df.columns:
        if col in cat_features:
            df[col] = df[col].astype('category')

    return df

In [ ]:
train = preprocess(train, is_train=True)
test = preprocess(test)

In [ ]:
train_agg = preprocess(train_agg, is_train=True)
test_agg = preprocess(test_agg)

In [ ]:
gc.collect()

22

# Baseline

In [ ]:
def get_best_threshold(y_true, pred_probs):
    candidate_thresholds = np.arange(0, 1, 0.005)
    candidate_scores = [f1_score(y_true, (pred_probs >= t).astype('int')) 
                        for t in candidate_thresholds]
    best_threshold = candidate_thresholds[np.argmax(candidate_scores)]
    return best_threshold

In [ ]:
%%time
scores_f1 = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
X, y = train, TARGET
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.loc[val_idx], y.iloc[val_idx]

    model = XGBClassifier(
        objective='binary:logistic',
        tree_method='gpu_hist' if HAVE_GPU else 'hist',
        enable_categorical=HAVE_GPU,
        eval_metric='aucpr',
        early_stopping_rounds=100, 
        seed=SEED
    ) 
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=0
    )
    val_probs = model.predict_proba(X_val)[:, 1]
    best_threshold = get_best_threshold(y_val, val_probs)
    val_preds = (val_probs >= best_threshold).astype('int')
    score = f1_score(y_val, val_preds)
    scores_f1.append(score)
    print(f'Fold #{fold}: ({model.best_iteration} rounds) F1-score = {score:.5f}')
    _ = gc.collect()

print(f'\nAvg F1-score = {np.mean(scores_f1):.5f} +/- {np.std(scores_f1):.5f}\n')

Fold #0: (97 rounds) F1-score = 0.68277
Fold #1: (96 rounds) F1-score = 0.69898
Fold #2: (95 rounds) F1-score = 0.69250
Fold #3: (87 rounds) F1-score = 0.70275
Fold #4: (79 rounds) F1-score = 0.71790

Avg F1-score = 0.69898 +/- 0.01164

CPU times: user 29.7 s, sys: 362 ms, total: 30 s
Wall time: 32 s


# Hyperparameter tuning

In [ ]:
def objective(trial, data, base_params):

    scores = []
    X, y = data

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step=0.01),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        'gamma': trial.suggest_float('gamma', 0, 20, step=0.1), #complexity-control
        'alpha': trial.suggest_float('alpha', 0, 5, step=0.05), #L1-reg
        'lambda': trial.suggest_float('lambda', 1e-3, 1e5, log=True), #L2-reg
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.05),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0, step=0.05),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 5, step=0.05)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = XGBClassifier(**base_params, **param_grid)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        val_probs = model.predict_proba(X_val)[:, 1]
        best_threshold = get_best_threshold(y_val, val_probs)
        val_preds = (val_probs >= best_threshold).astype('int')
        scores.append(f1_score(y_val, val_preds))

    return np.mean(scores)

In [ ]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True
    )
    
    return study

# Cross-validation and experiment setup

In [ ]:
def evaluate_model(data, model_params, verbose=True):
    oof_probs = {}  #out-of-fold predicted probabilities on train set
    test_preds = {} #predictions on test set for each fold after threshold-tuning
    scores_f1 = [] #F1 scores on validation set

    X, X_test, y = data

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = XGBClassifier(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        val_probs = model.predict_proba(X_val)[:, 1]
        oof_probs.update(dict(zip(val_idx, val_probs)))

        best_threshold = get_best_threshold(y_val, val_probs)
        val_preds = (val_probs >= best_threshold).astype('int')
        f1 = f1_score(y_val, val_preds)
        scores_f1.append(f1)
        if verbose:
            print(f'Fold #{fold}: ({model.best_iteration} rounds) F1 = {f1:.5f}')
        
        test_probs = model.predict_proba(X_test)[:, 1]
        test_preds[f'fold{fold}'] = (test_probs >= best_threshold).astype('int')
        
        _ = gc.collect()

    print(f'\nAvg F1 = {np.mean(scores_f1):.5f} +/- {np.std(scores_f1):.5f}')
    
    oof_probs = pd.Series(oof_probs).sort_index()
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mode'] = test_preds.mode(axis=1)[0]

    return oof_probs, test_preds

In [ ]:
def run_experiment(data, n_trials=5):
        
    X, X_test, y = data
    
    base_params = {
        'objective': 'binary:logistic',
        'n_estimators': 10000,
        'booster': 'gbtree',
        'eval_metric': 'aucpr',
        'early_stopping_rounds': 100,
        'tree_method': 'gpu_hist' if HAVE_GPU else 'hist',
        'predictor': 'gpu_predictor' if HAVE_GPU else 'cpu_predictor',
        'enable_categorical': HAVE_GPU,
        'verbosity': 1,
        'seed': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='maximize'
    )
    print(f'Best trial: {study.best_trial.number} -> Best value(F1): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    oof_probs, test_preds = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params
    )
    return oof_probs, test_preds

In [ ]:
oof_probs = pd.DataFrame()

### Generating submission files

In [ ]:
NOTEBOOK = '02'
SUBMISSION_PATH = f'/content/drive/MyDrive/data_science_competitions/zindi/landslide_prevention/submissions/nb_{NOTEBOOK}'
if not os.path.isdir(SUBMISSION_PATH):
    os.makedirs(SUBMISSION_PATH)

In [ ]:
def create_submission_files(test_preds: pd.DataFrame, expt_num: int):
    for col in (test_preds.columns):
        sub = sample_sub.copy()
        sub['Label'] = test_preds[col]
        sub.to_csv(f'{SUBMISSION_PATH}/{expt_num}_{col}.csv', index=False)

### Test experiment

In [ ]:
%%time
o, t = run_experiment(
    data=(train, test, TARGET),
    n_trials=5
)

[I 2022-09-30 14:43:08,795] A new study created in memory with name: no-name-65ee3dab-26ac-4b54-acd9-7a4311ce5ee3


---------------Hyperparameter tuning---------------


[I 2022-09-30 14:43:26,139] Trial 0 finished with value: 0.6691674999207106 and parameters: {'learning_rate': 0.16, 'max_depth': 12, 'min_child_weight': 16, 'gamma': 5.6000000000000005, 'alpha': 1.1, 'lambda': 308.87067834937415, 'subsample': 0.55, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.8, 'colsample_bynode': 0.7, 'scale_pos_weight': 1.0}. Best is trial 0 with value: 0.6691674999207106.
[I 2022-09-30 14:43:34,152] Trial 1 finished with value: 0.6102823903305248 and parameters: {'learning_rate': 0.27, 'max_depth': 11, 'min_child_weight': 7, 'gamma': 11.8, 'alpha': 4.9, 'lambda': 5764.3531133041315, 'subsample': 0.5, 'colsample_bytree': 0.65, 'colsample_bylevel': 0.65, 'colsample_bynode': 0.95, 'scale_pos_weight': 3.5}. Best is trial 0 with value: 0.6691674999207106.
[I 2022-09-30 14:44:09,898] Trial 2 finished with value: 0.702952279400187 and parameters: {'learning_rate': 0.04, 'max_depth': 3, 'min_child_weight': 19, 'gamma': 2.8000000000000003, 'alpha': 2.1, 'lambda': 0.59144

Best trial: 2 -> Best value(F1): 0.70295
Best hyperparameters:
learning_rate        - 0.04
max_depth            - 3
min_child_weight     - 19
gamma                - 2.8000000000000003
alpha                - 2.1
lambda               - 0.5914465750030369
subsample            - 0.95
colsample_bytree     - 0.7
colsample_bylevel    - 0.95
colsample_bynode     - 0.8500000000000001
scale_pos_weight     - 1.45
-----------------Cross-validation------------------
Fold #0: (667 rounds) F1 = 0.68332
Fold #1: (800 rounds) F1 = 0.70545
Fold #2: (746 rounds) F1 = 0.70037
Fold #3: (1144 rounds) F1 = 0.70427
Fold #4: (579 rounds) F1 = 0.72134

Avg F1 = 0.70295 +/- 0.01215
CPU times: user 2min 7s, sys: 709 ms, total: 2min 8s
Wall time: 2min 3s


### Experiment 1: Original features

In [ ]:
%%time
oof_probs['2_1'], test_preds_1 = run_experiment(
    data=(train, test, TARGET),
    n_trials=100
)

[I 2022-09-30 14:45:12,725] A new study created in memory with name: no-name-5b59fb3f-cffb-48f4-a617-968f9dc9535a


---------------Hyperparameter tuning---------------


[I 2022-09-30 14:45:29,907] Trial 0 finished with value: 0.6691674999207106 and parameters: {'learning_rate': 0.16, 'max_depth': 12, 'min_child_weight': 16, 'gamma': 5.6000000000000005, 'alpha': 1.1, 'lambda': 308.87067834937415, 'subsample': 0.55, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.8, 'colsample_bynode': 0.7, 'scale_pos_weight': 1.0}. Best is trial 0 with value: 0.6691674999207106.
[I 2022-09-30 14:45:37,804] Trial 1 finished with value: 0.6102823903305248 and parameters: {'learning_rate': 0.27, 'max_depth': 11, 'min_child_weight': 7, 'gamma': 11.8, 'alpha': 4.9, 'lambda': 5764.3531133041315, 'subsample': 0.5, 'colsample_bytree': 0.65, 'colsample_bylevel': 0.65, 'colsample_bynode': 0.95, 'scale_pos_weight': 3.5}. Best is trial 0 with value: 0.6691674999207106.
[I 2022-09-30 14:46:13,495] Trial 2 finished with value: 0.702952279400187 and parameters: {'learning_rate': 0.04, 'max_depth': 3, 'min_child_weight': 19, 'gamma': 2.8000000000000003, 'alpha': 2.1, 'lambda': 0.59144

In [ ]:
create_submission_files(test_preds_1, 1)

### Experiment 2: Created features

In [ ]:
created_features = [f for f in test_agg.columns 
                    if f.startswith('agg_') or f.startswith('grad_')]

In [ ]:
%%time
oof_probs['2_2'], test_preds_2 = run_experiment(
    data=(train_agg[created_features], test_agg[created_features], TARGET),
    n_trials=100
)

In [ ]:
create_submission_files(test_preds_2, 2)

### Experiment 3: All features

In [ ]:
%%time
oof_probs['2_3'], test_preds_3 = run_experiment(
    data=(train_agg, test_agg, TARGET),
    n_trials=100
)

In [ ]:
create_submission_files(test_preds_3, 3)

### Storing OOF predictions

In [ ]:
oof_probs.to_csv(f'{SUBMISSION_PATH}/oof_probs.csv', index=False)